In [ ]:
%pip install ipython-autotime
%pip install pandas
%pip install numpy
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np

time: 5.74 s (started: 2023-05-04 22:06:44 +07:00)


# Read dataset in JSON

In [4]:
dafi = pd.read_json('titanic_json.json')
dafi.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


time: 156 ms (started: 2023-05-04 22:10:27 +07:00)


In [5]:
dafi.shape

(891, 12)

time: 0 ns (started: 2023-05-04 22:10:30 +07:00)


# Preprocessing

Kolom yang akan menjadi prediktor adalah Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Cabin, dan Embarked.

Kolom respon adalah Survived.

Kolom dengan data non-angka akan diubah menjadi angka dengan teknik categorical label encoding.

Sebagai contoh, Sex:male diberi label 1, Sex:female diberi label 2.

Missing value pada kolom kategorik (Pclass, Sex, SibSp, Parch, Cabin, dan Embarked) akan diisi dengan nilai modus.

Missing value pada kolom numerik (Fare, Age) akan diisi dengan median.

#### Pisahkan X dan y

In [6]:
# Pilih variabel X yang sudah dalam bentuk angka
X_angka = dafi[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

# Pilih variabel X yang masih bukan angka
X_yang_bukan_angka = dafi[['Sex', 'Cabin', 'Embarked']]

# Pilih variabel Y
y = dafi['Survived']

time: 16 ms (started: 2023-05-04 22:10:32 +07:00)


### Ubah kolom non-angka ke dalam angka dengan label encoding

Kolom yg ingin kita ubah adalah Sex, Cabin, dan Embarked.

In [7]:
from sklearn.preprocessing import LabelEncoder
X_yang_bukan_angka_udah_di_encode = X_yang_bukan_angka.apply(LabelEncoder().fit_transform)
X_yang_bukan_angka_udah_di_encode

,Sex,Cabin,Embarked
0,1,147,2
1,0,81,0
2,0,147,2
3,0,55,2
4,1,147,2
...,...,...,...
886,1,147,2
887,0,30,2
888,0,147,2
889,1,60,0


time: 7.28 s (started: 2023-05-04 22:10:34 +07:00)


In [8]:
# Gabung hasilnya menjadi satu kesatuan yang utuh
X = pd.concat([X_angka, X_yang_bukan_angka_udah_di_encode], axis = 1)
X

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22.0,1,0,7.2500,1,147,2
1,1,38.0,1,0,71.2833,0,81,0
2,3,26.0,0,0,7.9250,0,147,2
3,1,35.0,1,0,53.1000,0,55,2
4,3,35.0,0,0,8.0500,1,147,2
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,147,2
887,1,19.0,0,0,30.0000,0,30,2
888,3,NaN,1,2,23.4500,0,147,2
889,1,26.0,0,0,30.0000,1,60,0


time: 63 ms (started: 2023-05-04 22:10:41 +07:00)


### Observasi ada tidaknya missing value

In [9]:
X.isna().sum()

Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex           0
Cabin         0
Embarked      0
dtype: int64

time: 15 ms (started: 2023-05-04 22:10:49 +07:00)


### Isi missing value kolom angka (Fare, Age) dengan median

In [10]:
X['Fare'].fillna(X['Fare'].median(), inplace = True)
X['Age'].fillna(X['Age'].median(), inplace = True)
X

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22.0,1,0,7.2500,1,147,2
1,1,38.0,1,0,71.2833,0,81,0
2,3,26.0,0,0,7.9250,0,147,2
3,1,35.0,1,0,53.1000,0,55,2
4,3,35.0,0,0,8.0500,1,147,2
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,147,2
887,1,19.0,0,0,30.0000,0,30,2
888,3,28.0,1,2,23.4500,0,147,2
889,1,26.0,0,0,30.0000,1,60,0


time: 31 ms (started: 2023-05-04 22:10:53 +07:00)


### Isi missing value kolom kategorik dengan modus

In [11]:
X['Pclass'].fillna(X['Pclass'].mode(), inplace = True)
X['SibSp'].fillna(X['SibSp'].mode(), inplace = True)
X['Parch'].fillna(X['Parch'].mode(), inplace = True)
X['Sex'].fillna(X['Sex'].mode(), inplace = True)
X['Cabin'].fillna(X['Cabin'].mode(), inplace = True)
X['Embarked'].fillna(X['Embarked'].mode(), inplace = True)
X

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22.0,1,0,7.2500,1,147,2
1,1,38.0,1,0,71.2833,0,81,0
2,3,26.0,0,0,7.9250,0,147,2
3,1,35.0,1,0,53.1000,0,55,2
4,3,35.0,0,0,8.0500,1,147,2
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,147,2
887,1,19.0,0,0,30.0000,0,30,2
888,3,28.0,1,2,23.4500,0,147,2
889,1,26.0,0,0,30.0000,1,60,0


time: 47 ms (started: 2023-05-04 22:11:05 +07:00)


In [12]:
X.isna().sum()

Pclass      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex         0
Cabin       0
Embarked    0
dtype: int64

time: 16 ms (started: 2023-05-04 22:11:08 +07:00)


# Train Test Split

Setelah data 'bersih', saatnya kita bikin model sesuai kreativitas masing2

In [13]:
from sklearn.model_selection import train_test_split
x_kereta, x_ujian, y_kereta, y_ujian = train_test_split(X, y, test_size = 0.23923, random_state = 455518)

time: 484 ms (started: 2023-05-04 22:16:22 +07:00)


# Evaluation Function

In [18]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
def evaluate_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(classification_report(y_true, y_pred))
    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

time: 0 ns (started: 2023-05-04 22:29:14 +07:00)


# Model

In [15]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.linear_model import LogisticRegressionCV

time: 1.44 s (started: 2023-05-04 22:28:48 +07:00)


## Support Vector Machine

In [19]:
svm_clf = SVC()
svm_clf.fit(x_kereta, y_kereta)
svm_pred = svm_clf.predict(x_ujian)
evaluate_classification(y_ujian, svm_pred)

              precision    recall  f1-score   support

           0       0.70      0.96      0.81       134
           1       0.81      0.33      0.46        80

    accuracy                           0.72       214
   macro avg       0.76      0.64      0.64       214
weighted avg       0.74      0.72      0.68       214



{'Accuracy': 0.719626168224299,
 'Precision': 0.8125,
 'Recall': 0.325,
 'F1-score': 0.46428571428571436}

time: 109 ms (started: 2023-05-04 22:29:17 +07:00)


## Decision Tree Classifier

In [20]:
dt_clf = DecisionTreeClassifier(max_depth=5)
dt_clf.fit(x_kereta, y_kereta)
dt_pred = dt_clf.predict(x_ujian)
evaluate_classification(y_ujian, dt_pred)

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       134
           1       0.88      0.65      0.75        80

    accuracy                           0.84       214
   macro avg       0.85      0.80      0.81       214
weighted avg       0.84      0.84      0.83       214



{'Accuracy': 0.8364485981308412,
 'Precision': 0.8813559322033898,
 'Recall': 0.65,
 'F1-score': 0.7482014388489208}

time: 63 ms (started: 2023-05-04 22:29:19 +07:00)


## K Neighbors Classifier

In [21]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(x_kereta, y_kereta)
knn_pred = knn_clf.predict(x_ujian)
evaluate_classification(y_ujian, knn_pred)

              precision    recall  f1-score   support

           0       0.74      0.81      0.77       134
           1       0.62      0.54      0.58        80

    accuracy                           0.71       214
   macro avg       0.68      0.67      0.68       214
weighted avg       0.70      0.71      0.70       214



{'Accuracy': 0.705607476635514,
 'Precision': 0.6231884057971014,
 'Recall': 0.5375,
 'F1-score': 0.5771812080536913}

time: 47 ms (started: 2023-05-04 22:29:23 +07:00)


## Ada Boost Classifier

In [22]:
ada_clf = AdaBoostClassifier(n_estimators=100)
ada_clf.fit(x_kereta, y_kereta)
ada_pred = ada_clf.predict(x_ujian)
evaluate_classification(y_ujian, ada_pred)

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       134
           1       0.81      0.75      0.78        80

    accuracy                           0.84       214
   macro avg       0.83      0.82      0.83       214
weighted avg       0.84      0.84      0.84       214



{'Accuracy': 0.8411214953271028,
 'Precision': 0.8108108108108109,
 'Recall': 0.75,
 'F1-score': 0.7792207792207791}

time: 422 ms (started: 2023-05-04 22:29:25 +07:00)


## MLP Classifier

In [23]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=1000)
mlp_clf.fit(x_kereta, y_kereta)
mlp_pred = mlp_clf.predict(x_ujian)
evaluate_classification(y_ujian, mlp_pred)

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       134
           1       0.74      0.69      0.71        80

    accuracy                           0.79       214
   macro avg       0.78      0.77      0.78       214
weighted avg       0.79      0.79      0.79       214



{'Accuracy': 0.794392523364486,
 'Precision': 0.7432432432432432,
 'Recall': 0.6875,
 'F1-score': 0.7142857142857143}

time: 3.19 s (started: 2023-05-04 22:29:27 +07:00)


# Gaussian Naive Bayes

In [24]:
gnb_clf = GaussianNB()
gnb_clf.fit(x_kereta, y_kereta)
gnb_pred = gnb_clf.predict(x_ujian)
evaluate_classification(y_ujian, gnb_pred)

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       134
           1       0.73      0.64      0.68        80

    accuracy                           0.78       214
   macro avg       0.76      0.75      0.75       214
weighted avg       0.77      0.78      0.77       214



{'Accuracy': 0.7757009345794392,
 'Precision': 0.7285714285714285,
 'Recall': 0.6375,
 'F1-score': 0.68}

time: 47 ms (started: 2023-05-04 22:29:31 +07:00)


# Random Forest

In [25]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf_clf.fit(x_kereta, y_kereta)
rf_pred = rf_clf.predict(x_ujian)
evaluate_classification(y_ujian, rf_pred)

              precision    recall  f1-score   support

           0       0.80      0.95      0.87       134
           1       0.87      0.60      0.71        80

    accuracy                           0.82       214
   macro avg       0.84      0.77      0.79       214
weighted avg       0.83      0.82      0.81       214



{'Accuracy': 0.8177570093457944,
 'Precision': 0.8727272727272727,
 'Recall': 0.6,
 'F1-score': 0.711111111111111}

time: 610 ms (started: 2023-05-04 22:29:34 +07:00)


# Quadratic Discriminant Analysis

In [26]:
qda_clf = QuadraticDiscriminantAnalysis()
qda_clf.fit(x_kereta, y_kereta)
qda_pred = qda_clf.predict(x_ujian)
evaluate_classification(y_ujian, qda_pred)

              precision    recall  f1-score   support

           0       0.84      0.89      0.87       134
           1       0.79      0.72      0.76        80

    accuracy                           0.83       214
   macro avg       0.82      0.81      0.81       214
weighted avg       0.83      0.83      0.83       214



{'Accuracy': 0.8271028037383178,
 'Precision': 0.7945205479452054,
 'Recall': 0.725,
 'F1-score': 0.7581699346405228}

time: 62 ms (started: 2023-05-04 22:29:36 +07:00)


# Logistic Regression CV

In [27]:
lr_cv_clf = LogisticRegressionCV(cv=5, max_iter=1000)
lr_cv_clf.fit(x_kereta, y_kereta)
lr_cv_pred = lr_cv_clf.predict(x_ujian)
evaluate_classification(y_ujian, lr_cv_pred)

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       134
           1       0.77      0.68      0.72        80

    accuracy                           0.80       214
   macro avg       0.80      0.78      0.78       214
weighted avg       0.80      0.80      0.80       214



{'Accuracy': 0.8037383177570093,
 'Precision': 0.7714285714285715,
 'Recall': 0.675,
 'F1-score': 0.72}

time: 2.12 s (started: 2023-05-04 22:29:38 +07:00)


# Label Propagation

In [28]:
lp_clf = LabelPropagation(kernel='rbf', gamma=0.1)
lp_clf.fit(x_kereta, y_kereta)
lp_pred = lp_clf.predict(x_ujian)
evaluate_classification(y_ujian, lp_pred)

              precision    recall  f1-score   support

           0       0.73      0.87      0.79       134
           1       0.68      0.47      0.56        80

    accuracy                           0.72       214
   macro avg       0.71      0.67      0.68       214
weighted avg       0.71      0.72      0.71       214



{'Accuracy': 0.719626168224299,
 'Precision': 0.6785714285714286,
 'Recall': 0.475,
 'F1-score': 0.5588235294117648}

time: 94 ms (started: 2023-05-04 22:29:42 +07:00)


# Label Spreading

In [29]:
ls_clf = LabelSpreading(kernel='knn', n_neighbors=5, max_iter=1000)
ls_clf.fit(x_kereta, y_kereta)
ls_pred = ls_clf.predict(x_ujian)
evaluate_classification(y_ujian, ls_pred)

              precision    recall  f1-score   support

           0       0.74      0.81      0.77       134
           1       0.62      0.53      0.57        80

    accuracy                           0.70       214
   macro avg       0.68      0.67      0.67       214
weighted avg       0.69      0.70      0.70       214



{'Accuracy': 0.7009345794392523,
 'Precision': 0.6176470588235294,
 'Recall': 0.525,
 'F1-score': 0.5675675675675677}

time: 109 ms (started: 2023-05-04 22:29:45 +07:00)


# All Model

In [32]:
# Model
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# Metrics evaluasi
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def auto_model(x_kereta, y_kereta, x_ujian, y_ujian):
  # Tulis semua model yg mau dibikin jadi sebuah list kayak gini
  daftar_model_kita = [SVC(), DecisionTreeClassifier(), KNeighborsClassifier(), AdaBoostClassifier(), MLPClassifier(), GaussianNB(), RandomForestClassifier(), QuadraticDiscriminantAnalysis(), LabelPropagation(), LabelSpreading(), LogisticRegressionCV()]

  # Siapin list buat nyimpen nilai metrics nya, bebas sih mau pilih apa aja
  akurasi = []
  formula_1 = []
  temennya_akurasi = []
  recall_recall = []

  # Looping deh 
  for model in daftar_model_kita:
    # Di fit
    model.fit(x_kereta, y_kereta)

    # Prediksi dataset ujian
    prediksi_model_kita = model.predict(x_ujian)
    akurasi.append(accuracy_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    formula_1.append(f1_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    temennya_akurasi.append(precision_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    recall_recall.append(recall_score(y_true = y_ujian, y_pred = prediksi_model_kita))

  # Jadiin sebuah dataframe biar keren gituloh
  hasil = {
    'model' : daftar_model_kita,
    'akurasi' : akurasi,
    'formula_1' : formula_1,
    'temennya_akurasi' : temennya_akurasi,
    'recall_recall' : recall_recall
  }

  # Balikin hasilnya
  return(pd.DataFrame(hasil))

time: 0 ns (started: 2023-05-04 22:32:38 +07:00)


In [ ]:
# Saatnya kita uji
hasilnya_masbro = auto_model(x_kereta, y_kereta, x_ujian, y_ujian)

In [37]:
hasilnya_masbro.sort_values(by='akurasi', ascending=False)

,model,akurasi,formula_1,temennya_akurasi,recall_recall
3,"(DecisionTreeClassifier(max_depth=1, random_st...",0.845794,0.784314,0.821918,0.7500
6,"(DecisionTreeClassifier(max_features='sqrt', r...",0.831776,0.760000,0.814286,0.7125
7,QuadraticDiscriminantAnalysis(),0.827103,0.758170,0.794521,0.7250
4,MLPClassifier(),0.808411,0.738854,0.753247,0.7250
10,LogisticRegressionCV(),0.799065,0.715232,0.760563,0.6750
1,DecisionTreeClassifier(),0.789720,0.693878,0.761194,0.6375
5,GaussianNB(),0.775701,0.680000,0.728571,0.6375
0,SVC(),0.719626,0.464286,0.812500,0.3250
2,KNeighborsClassifier(),0.705607,0.577181,0.623188,0.5375
9,LabelSpreading(),0.668224,0.517007,0.567164,0.4750


time: 234 ms (started: 2023-05-04 22:35:27 +07:00)


### Bonus content: gunakan Cross Validation, tidak perlu pakai train_test_split!

Tujuannya pakai CV sesuai namanya, yaitu untuk validasi hasil akhir. 

Misal kita sudah punya 2 kandidat model, yaitu Decision Tree vs SVC. Secara akurasi bagusan Decision Tree, tapi secara F1 score bagusan SVC. 

Untuk memastikan, kita bisa lihat rata2 nilai cross validationnya. CV itu intinya ngecek model kita berulang kali (bukan cuma 1 kali).

In [34]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# Ini ni salah satunya
from sklearn.model_selection import KFold, cross_val_score

def auto_model_pake_cv(X, y):
  # Tulis semua model yg mau dibikin jadi sebuah list kayak gini
  daftar_model_kita = [SVC(), DecisionTreeClassifier(), KNeighborsClassifier(), AdaBoostClassifier(), MLPClassifier(), GaussianNB(), RandomForestClassifier(), QuadraticDiscriminantAnalysis(), LabelPropagation(), LabelSpreading(), LogisticRegressionCV()]

  # Bikin CV buat daftar kerja
  # CV ini nanti otomatis dipakai oleh model
  cv_kita = KFold(n_splits = 3)

  # Siapin list buat nyimpen nilai metrics nya, bebas sih mau pilih apa aja
  akurasi = []
  formula_1 = []
  temennya_akurasi = []
  recall_recall = []

  # Looping deh 
  for model in daftar_model_kita:
    # Ga usah fit & predict2 lagi bang, langsung kita hitung metricsnya
    akurasi.append(np.mean(cross_val_score(model, X, y, cv = cv_kita, scoring = 'accuracy')))
    formula_1.append(np.mean(cross_val_score(model, X, y, cv = cv_kita, scoring = 'f1')))
    temennya_akurasi.append(np.mean(cross_val_score(model, X, y, cv = cv_kita, scoring = 'precision')))
    recall_recall.append(np.mean(cross_val_score(model, X, y, cv = cv_kita, scoring = 'recall')))

  # Jadiin sebuah dataframe biar keren gituloh
  hasil = {
    'model' : daftar_model_kita,
    'akurasi' : akurasi,
    'formula_1' : formula_1,
    'temennya_akurasi' : temennya_akurasi,
    'recall_recall' : recall_recall
  }

  # Balikin hasilnya
  return(pd.DataFrame(hasil))

time: 0 ns (started: 2023-05-04 22:32:52 +07:00)


In [ ]:
hasilnya_pake_cv_masbro = auto_model_pake_cv(X, y)

In [38]:
hasilnya_pake_cv_masbro.sort_values(by='akurasi', ascending=False)

,model,akurasi,formula_1,temennya_akurasi,recall_recall
6,RandomForestClassifier(),0.801347,0.722871,0.726772,0.714011
3,AdaBoostClassifier(),0.789001,0.726190,0.727422,0.728095
7,QuadraticDiscriminantAnalysis(),0.787879,0.721571,0.724931,0.718603
10,LogisticRegressionCV(),0.787879,0.714076,0.738239,0.692388
1,DecisionTreeClassifier(),0.781145,0.707262,0.710144,0.714801
5,GaussianNB(),0.769921,0.685857,0.708064,0.666341
4,MLPClassifier(),0.764310,0.705752,0.721887,0.649502
2,KNeighborsClassifier(),0.728395,0.617428,0.663221,0.578587
0,SVC(),0.679012,0.439393,0.667091,0.327654
8,LabelPropagation(),0.662177,0.461168,0.596533,0.377812


time: 47 ms (started: 2023-05-04 22:35:39 +07:00)


### Bonus content

Model di bawah ini juga buat cross validation, cuma lebih sophisticated (lebih rumit utk dijelaskan~)

CalibratedClassifier bisa digunakan untuk meningkatkan performa model apapun yg kita punya

RidgeClassifierCV khusus buat ridge regression (otomatis)

In [42]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifierCV

def auto_model_2(x_kereta, y_kereta, x_ujian, y_ujian):
  # Tulis semua model yg mau dibikin jadi sebuah list kayak gini
  daftar_model_kita = [SVC(), DecisionTreeClassifier(), KNeighborsClassifier(), AdaBoostClassifier(), MLPClassifier(), GaussianNB(), RandomForestClassifier(), QuadraticDiscriminantAnalysis(), LabelPropagation(), LabelSpreading(), LogisticRegressionCV()]

  # Siapin list buat nyimpen nilai metrics nya, bebas sih mau pilih apa aja
  akurasi = []
  akurasi_kalibrasi = []
  akurasi_jembatan = []

  # Looping deh 
  for model in daftar_model_kita:
    # Di fit
    model.fit(x_kereta, y_kereta)
    
    # Di kalibrasi & jembatani
    model_kalibrasi = CalibratedClassifierCV(model, cv = 'prefit').fit(x_kereta, y_kereta)
    model_jembatan = RidgeClassifierCV(alphas = [0.3, 1.9, 4.5], cv = 3).fit(x_kereta, y_kereta)

    # Prediksi dataset ujian setelah di kalibrasi
    prediksi_model_kita = model.predict(x_ujian)
    prediksi_model_kalibrasi = model_kalibrasi.predict(x_ujian)
    prediksi_model_jembatan = model_jembatan.predict(x_ujian)

    akurasi.append(accuracy_score(y_true = y_ujian, y_pred = prediksi_model_kita))
    akurasi_kalibrasi.append(accuracy_score(y_true = y_ujian, y_pred = prediksi_model_kalibrasi))
    akurasi_jembatan.append(accuracy_score(y_true = y_ujian, y_pred = prediksi_model_jembatan))

  # Jadiin sebuah dataframe biar keren gituloh
  hasil = {
    'model' : daftar_model_kita,
    'akurasi' : akurasi,
    'akurasi_kalibrasi' : akurasi_kalibrasi,
    'akurasi_jembatan' : akurasi_jembatan
  }

  # Balikin hasilnya
  return(pd.DataFrame(hasil))

time: 16 ms (started: 2023-05-04 22:38:33 +07:00)


In [ ]:
hasillagi = auto_model_2(x_kereta, y_kereta, x_ujian, y_ujian)

In [44]:
hasillagi.sort_values(by='akurasi', ascending=False)

,model,akurasi,akurasi_kalibrasi,akurasi_jembatan
3,"(DecisionTreeClassifier(max_depth=1, random_st...",0.845794,0.845794,0.799065
7,QuadraticDiscriminantAnalysis(),0.827103,0.766355,0.799065
6,"(DecisionTreeClassifier(max_features='sqrt', r...",0.817757,0.817757,0.799065
1,DecisionTreeClassifier(),0.799065,0.799065,0.799065
4,MLPClassifier(),0.799065,0.789720,0.799065
10,LogisticRegressionCV(),0.799065,0.799065,0.799065
5,GaussianNB(),0.775701,0.771028,0.799065
0,SVC(),0.719626,0.724299,0.799065
2,KNeighborsClassifier(),0.705607,0.705607,0.799065
9,LabelSpreading(),0.668224,0.640187,0.799065


time: 187 ms (started: 2023-05-04 22:38:41 +07:00)
